In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import ast

from sys import path
path.insert(0, "lib")
from lib.onedmodel import *
from pathlib import Path

In [2]:
# Plotting settings
plt.style.use('ggplot')
fig_width = 25  # figure width in cm
inches_per_cm = 0.393701               # Convert cm to inch
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width*inches_per_cm  # width in inches
fig_height = fig_width*golden_mean      # height in inches
fig_size =  [fig_width,fig_height]
label_size = 14
tick_size = 12
params = {'backend': 'ps',
          'lines.linewidth': 1.5,
          'axes.labelsize': label_size,
          'axes.titlesize': label_size,
          'font.size': label_size,
          'legend.fontsize': tick_size,
          'xtick.labelsize': tick_size,
          'ytick.labelsize': tick_size,
          'text.usetex': False,
          'figure.figsize': fig_size,
          "font.weight": "bold",
          "axes.labelweight": "bold"}
plt.rcParams.update(params)

In [3]:
# Initialise
nSGD = 10**3
nsamples = 10**3
shuffle = True
batch_size= 20
w0 = 1.8
seed_init = 1
seed_data = 1
nw0 = 2
wmax = nw0 * w0
frac_max = 10**-2
model = PolyModel(w0=w0, seed=seed_init, wmax=wmax, wmin=-wmax)
sgd_runner = SGDPolyRunner(nSGD=nSGD, nsamples=nsamples, seed=seed_data, shuffle=shuffle, 
                           batch_size=batch_size, auto=True)
# Run parameter sweep
w0_range = np.arange(1.5, 2, 0.1)
batch_range = np.arange(5, 25, 10)
lr_range = [0.1, 0.01, 0.001]
df = sgd_runner.parameter_sweep(w0_range, batch_range, lr_range, 
                        model, frac_max=frac_max)

Running experiment 0 over 30
Running experiment 1 over 30
Running experiment 2 over 30
Running experiment 3 over 30
Running experiment 4 over 30
Running experiment 5 over 30
Running experiment 6 over 30


/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


Running experiment 7 over 30
Running experiment 8 over 30
Running experiment 9 over 30


/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


Running experiment 10 over 30
Running experiment 11 over 30
Running experiment 12 over 30
Running experiment 13 over 30
Running experiment 14 over 30
Running experiment 15 over 30


...

Running experiment 22 over 30
Running experiment 23 over 30
Running experiment 24 over 30


/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


Running experiment 25 over 30
Running experiment 26 over 30
Running experiment 27 over 30


/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/guime/anaconda3/envs/devinterp/lib/python3.11/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


Running experiment 28 over 30
Running experiment 29 over 30


In [15]:
cleaned_df = df.copy().dropna()
for i, w0 in enumerate(w0_range):
    escape_rate = cleaned_df["escape_rate"].loc[cleaned_df["w0"]==w0].to_numpy()
    intercept = cleaned_df["intercept"].loc[cleaned_df["w0"]==w0].to_numpy()
    fraction = cleaned_df["fraction"].loc[cleaned_df["w0"]==w0].to_numpy()
    plt.subplot(3,2,i+1)
    for j in range(len(escape_rate)):
        log_frac = np.log(fraction[j])
        print(log_frac)
        time = np.arange(0,1, len(fraction[j]))
        y = escape_rate[j] * time + intercept[j]
        plt.plot(time, y)
        plt.plot(time, log_frac, '-')
        plt.xlabel("time")
        plt.ylabel("Log fraction")
plt.show()

[-2.86738224 -3.56052942 -4.57213033 -4.8598124  -5.26527751 -5.95842469
 -5.95842469        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf        -inf        -inf
        -inf        -inf        -inf        -inf   

<ipython-input-15-7faf7255f6b3>:8: RuntimeWarning: divide by zero encountered in log
  log_frac = np.log(fraction[j])


ValueError: x and y must have same first dimension, but have shapes (1,) and (201,)